<a href="https://colab.research.google.com/github/aralzaabi1991-coder/Training-ENEC/blob/main/first_python_chatbot_logo_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install openai


In [5]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("openai")


In [4]:
from openai import OpenAI

client = OpenAI()
#your system prompt -playground
SYSTEM_PROMPT = """
"Create a Python chatbot that can answer ONLY questions about gold and silver prices. The chatbot should reject or politely refuse to answer any other type of question. Persist in following these requirements for every interaction.\n\nWhen generating or evaluating a reply:\n- First, determine if the user's query is specifically about gold or silver prices.\n  - Use internal reasoning to decide if the input is about gold/silver prices (e.g., \"What's the price of gold?\" is valid; \"What is bitcoin's price?\" is not.)\n  - If YES: Produce a helpful, accurate, and concise answer about the current or recent price of gold or silver, as appropriate.\n  - If NO: Respond politely that you can answer only questions about gold and silver prices, and decline to answer any other queries.\n- NEVER answer non-gold/silver price questions, even if tangential.\n- For ambiguous or borderline queries, explain that only direct questions about gold or silver prices are accepted and request clarification if needed.\n\nOutput Format:\n- Replies must be plain text. \n- The response should be 1-3 sentences maximum.\n\nReasoning and Conclusion ORDER:\n- Reason internally (not output to user) whether the query is about gold/silver prices.\n- Output only the appropriate response as the last step.\n\nExamples\n\nExample 1  \nInput: What is the current price of gold?  \nOutput: The current price of gold is approximately $[CURRENT_GOLD_PRICE] per ounce.\n\nExample 2  \nInput: How much is silver trading for today?  \nOutput: The current price of silver is about $[CURRENT_SILVER_PRICE] per ounce.\n\nExample 3  \nInput: Tell me a joke.  \nOutput: I'm sorry, I can only answer questions about gold and silver prices.\n\nExample 4  \nInput: What are the factors affecting gold's value?  \nOutput: I'm sorry, I can only provide the current prices of gold and silver, not other information.\n\nExample 5  \nInput: What's the price of platinum?  \nOutput: I'm sorry, I can only answer questions about gold and silver prices.\n\n(For real deployment, fill [CURRENT_GOLD_PRICE] and [CURRENT_SILVER_PRICE] with real-time data. The same format applies for all relevant queries.)\n\n---\nReminder: Only answer gold/silver price questions in 1-3 sentence plain text responses; decline all other requests. Reason internally before replying."

"""

messages = [
    {
        "role": "system",
        "content": [
            {"type": "input_text", "text": SYSTEM_PROMPT}
        ],
    }
]

print("Chatbot ready. Type 'exit' to stop.")

while True:
    user_text = input("You: ").strip()
    if not user_text:
        continue
    if user_text.lower() in {"exit", "quit", "bye"}:
        print("Bot: Goodbye!")
        break

    # Add user message
    messages.append(
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_text}
            ],
        }
    )

    # Call the Responses API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True,
        include=["web_search_call.action.sources"]
    )

    assistant_text = response.output_text

    print("Bot:", assistant_text)

    # Store assistant message to preserve context
    messages.append(
        {
            "role": "assistant",
            "content": [
                {"type": "output_text", "text": assistant_text}
            ],
        }
    )


Chatbot ready. Type 'exit' to stop.
You: where is dubai
Bot: I'm sorry, I can only answer questions about gold and silver prices.
You: exit
Bot: Goodbye!


In [7]:
pip install gradio

In [8]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """"Create a Python chatbot that can answer ONLY questions about gold and silver prices. The chatbot should reject or politely refuse to answer any other type of question. Persist in following these requirements for every interaction.\n\nWhen generating or evaluating a reply:\n- First, determine if the user's query is specifically about gold or silver prices.\n  - Use internal reasoning to decide if the input is about gold/silver prices (e.g., \"What's the price of gold?\" is valid; \"What is bitcoin's price?\" is not.)\n  - If YES: Produce a helpful, accurate, and concise answer about the current or recent price of gold or silver, as appropriate.\n  - If NO: Respond politely that you can answer only questions about gold and silver prices, and decline to answer any other queries.\n- NEVER answer non-gold/silver price questions, even if tangential.\n- For ambiguous or borderline queries, explain that only direct questions about gold or silver prices are accepted and request clarification if needed.\n\nOutput Format:\n- Replies must be plain text. \n- The response should be 1-3 sentences maximum.\n\nReasoning and Conclusion ORDER:\n- Reason internally (not output to user) whether the query is about gold/silver prices.\n- Output only the appropriate response as the last step.\n\nExamples\n\nExample 1  \nInput: What is the current price of gold?  \nOutput: The current price of gold is approximately $[CURRENT_GOLD_PRICE] per ounce.\n\nExample 2  \nInput: How much is silver trading for today?  \nOutput: The current price of silver is about $[CURRENT_SILVER_PRICE] per ounce.\n\nExample 3  \nInput: Tell me a joke.  \nOutput: I'm sorry, I can only answer questions about gold and silver prices.\n\nExample 4  \nInput: What are the factors affecting gold's value?  \nOutput: I'm sorry, I can only provide the current prices of gold and silver, not other information.\n\nExample 5  \nInput: What's the price of platinum?  \nOutput: I'm sorry, I can only answer questions about gold and silver prices.\n\n(For real deployment, fill [CURRENT_GOLD_PRICE] and [CURRENT_SILVER_PRICE] with real-time data. The same format applies for all relevant queries.)\n\n---\nReminder: Only answer gold/silver price questions in 1-3 sentence plain text responses; decline all other requests. Reason internally before replying."
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages

with gr.Blocks() as demo:
    gr.Markdown("# Python Chatbot (OpenAI Responses API)")

    chatbot = gr.Chatbot(height=420)
    msg = gr.Textbox(placeholder="Type your Python question and press Enter")
    clear = gr.Button("Clear")

    state = gr.State(init_messages())

    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    clear.click(
        fn=lambda: ([], init_messages()),
        inputs=None,
        outputs=[chatbot, state]
    )

demo.launch(debug=False)


/tmp/ipython-input-2383512617.py:60: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)
/tmp/ipython-input-2383512617.py:60: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=420)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4e6dd8dd64a7f5c4e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages


# -----------------------------
# Gradio UI (Upgraded)
# -----------------------------

FAQ_QUESTIONS = [
    "What is the difference between a list, tuple, and set in Python?",
    "How do I use dictionaries effectively in Python?",
    "What are Python functions and how do *args and **kwargs work?",
    "How does OOP work in Python (classes, objects, inheritance)?",
    "How do I handle errors using try/except?",
    "What are list comprehensions and when should I use them?",
    "How do I read and write files in Python?"
]

def set_question(q):
    return q

def clear_all():
    return [], init_messages(), ""

css = """
/* Overall spacing + subtle polish */
#app_container {max-width: 1200px; margin: 0 auto;}

/* Header styling */
.header-wrap {
    display: flex;
    align-items: center;
    gap: 14px;
    padding: 10px 6px 2px 6px;
}
.header-title {
    font-size: 28px;
    font-weight: 700;
    line-height: 1.1;
}
.header-subtitle {
    font-size: 12.5px;
    opacity: 0.75;
    margin-top: 2px;
}

/* FAQ card-like feel */
.faq-box {
    border: 1px solid rgba(255,255,255,0.08);
    border-radius: 12px;
    padding: 14px;
}

/* Make buttons full width in FAQ column */
.faq-btn button {
    width: 100%;
    justify-content: flex-start;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
    # Header Row
    with gr.Row():
        with gr.Column(scale=1, min_width=80):
            # Use your local file path or URL
            logo = gr.Image(
                value="https://github.com/Decoding-Data-Science/nov25/blob/main/logo_python.png",
                label=None,
                show_label=False,
                show_download_button=False,
                show_share_button=False,
                height=64,
                width=64,
                container=False
            )
        with gr.Column(scale=10):
            gr.HTML(
                """
                <div class="header-wrap">
                    <div>
                        <div class="header-title">Python Tutor Bot</div>
                        <div class="header-subtitle">
                            Ask anything about Python — concepts, debugging, best practices, and examples.
                        </div>
                    </div>
                </div>
                """
            )

    gr.Markdown("---")

    # State (unchanged logic)
    state = gr.State(init_messages())

    # Two-column layout
    with gr.Row(equal_height=True):
        # LEFT: FAQ + Quick Ask
        with gr.Column(scale=4, min_width=320):
            with gr.Group(elem_classes=["faq-box"]):
                gr.Markdown("### FAQ — Most Asked Python Questions")

                gr.Markdown(
                    "Click a question to auto-fill it, then press **Enter** or click **Send**."
                )

                # Buttons for FAQ
                faq_buttons = []
                for q in FAQ_QUESTIONS:
                    b = gr.Button(q, elem_classes=["faq-btn"])
                    faq_buttons.append(b)

                gr.Markdown("### Quick prompt ideas")
                quick = gr.Radio(
                    choices=[
                        "Explain with a simple example",
                        "Give me a beginner-friendly analogy",
                        "Show common mistakes to avoid",
                        "Provide a short quiz question",
                        "Compare two approaches briefly"
                    ],
                    label="Add a style preference (optional)",
                    value=None
                )

        # RIGHT: Chat area
        with gr.Column(scale=8, min_width=520):
            chatbot = gr.Chatbot(
                height=520,
                bubble_full_width=False,
                label="Conversation"
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your Python question here…",
                    label=None,
                    scale=9
                )
                send = gr.Button("Send", variant="primary", scale=1)

            with gr.Row():
                clear = gr.Button("Clear Chat")
                gr.Markdown(
                    "<span style='opacity:0.7;font-size:12px;'>Context is preserved across turns unless you clear.</span>"
                )

    # FAQ button -> fill textbox
    for b, q in zip(faq_buttons, FAQ_QUESTIONS):
        b.click(
            fn=lambda q=q: set_question(q),
            inputs=None,
            outputs=msg
        )

    # Optional quick preference: append hint to textbox (UI-only)
    def apply_quick_pref(pref, current_text):
        if not pref:
            return current_text
        if current_text and current_text.strip():
            return f"{current_text.strip()} ({pref})"
        return pref

    quick.change(
        fn=apply_quick_pref,
        inputs=[quick, msg],
        outputs=msg
    )

    # Submit logic (unchanged)
    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    send.click(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    # Clear
    clear.click(
        fn=clear_all,
        inputs=None,
        outputs=[chatbot, state, msg]
    )

demo.launch(debug=False)


/tmp/ipython-input-848837234.py:146: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-848837234.py:146: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-848837234.py:212: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-848837234.py:212: DeprecationWarning: The 'bubble_full_width' p

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://08b8d5896369f9d38f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

# RAW GitHub URL so it renders correctly
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def build_initial_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def chat_fn(user_message, history, messages_state):
    if messages_state is None:
        messages_state = build_initial_messages()

    # Add user message to state
    messages_state.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_message}]
        }
    )

    # Call Responses API
    resp = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages_state,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_message = resp.output_text

    # Add assistant message to state
    messages_state.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_message}]
        }
    )

    # Update visible chat
    history = history + [(user_message, assistant_message)]

    return "", history, messages_state

def reset_chat():
    return [], build_initial_messages()

with gr.Blocks() as demo:
    # Header with logo at native size (no width/height restriction)
    gr.Markdown(
        f"""
<div style="display:flex; align-items:center; gap:12px;">
  <img src="{LOGO_URL}" />
  <div>
    <div style="font-size:22px; font-weight:600;">Python Chatbot</div>
    <div style="font-size:13px; opacity:0.8;">Powered by OpenAI Responses API</div>
  </div>
</div>
"""
    )

    chatbot = gr.Chatbot(height=420)

    input_box = gr.Textbox(
        placeholder="Type your Python question and press Enter",
        show_label=False
    )

    clear_btn = gr.Button("Clear")

    messages_state = gr.State(build_initial_messages())

    input_box.submit(
        fn=chat_fn,
        inputs=[input_box, chatbot, messages_state],
        outputs=[input_box, chatbot, messages_state]
    )

    clear_btn.click(
        fn=reset_chat,
        inputs=None,
        outputs=[chatbot, messages_state]
    )

demo.launch(debug=False)


/tmp/ipython-input-2076293516.py:109: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)
/tmp/ipython-input-2076293516.py:109: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=420)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8dc6bcec5994819bf8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

# Use the RAW GitHub URL so the image loads correctly
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def build_initial_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def chat_fn(user_message, history, messages_state):
    # Initialize state if needed
    if messages_state is None:
        messages_state = build_initial_messages()

    # Append user input
    messages_state.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_message}]
        }
    )

    # Call Responses API
    resp = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages_state,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_message = resp.output_text

    # Append assistant output
    messages_state.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_message}]
        }
    )

    # Update visible chat history
    history = history + [(user_message, assistant_message)]

    # Clear textbox, update chat + state
    return "", history, messages_state

def reset_chat():
    return [], build_initial_messages()

css = """
/* Overall app container styling */
#app_container {
    max-width: 1200px;
    margin: 0 auto;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #1e1e2f, #2e2e4a); /* Dark blue-purple gradient background */
    color: #e0e0e0; /* Light text color for general content */
    border-radius: 15px;
    box-shadow: 0 8px 30px rgba(0, 0, 0, 0.4);
    overflow: hidden; /* Ensures background gradient is contained */
}

/* Header styling */
.header-wrap {
    display: flex;
    align-items: center;
    gap: 15px;
    padding: 20px 25px;
    background-color: rgba(0, 0, 0, 0.3); /* Slightly transparent dark background */
    border-bottom: 1px solid rgba(255, 255, 255, 0.1);
    margin-bottom: 20px;
}
.header-title {
    font-size: 32px;
    font-weight: 700;
    line-height: 1.2;
    color: #a0e0ff; /* Bright blue for title */
    text-shadow: 1px 1px 3px rgba(0,0,0,0.5);
}
.header-subtitle {
    font-size: 14px;
    opacity: 0.85;
    margin-top: 5px;
    color: #c0d0f0;
}

/* FAQ card-like feel */
.faq-box {
    border: 1px solid rgba(255,255,255,0.1);
    border-radius: 12px;
    padding: 18px;
    background-color: rgba(0, 0, 0, 0.2);
    box-shadow: 0 4px 15px rgba(0,0,0,0.3);
    margin-bottom: 20px;
}

/* Make buttons full width in FAQ column */
.faq-btn button {
    width: 100%;
    justify-content: flex-start;
    background-color: #3a3a5a; /* Darker blue-purple button */
    color: #f0f0f0;
    border: none;
    border-radius: 8px;
    padding: 12px 15px;
    margin-bottom: 8px;
    transition: background-color 0.2s ease;
}
.faq-btn button:hover {
    background-color: #4a4a7a;
    transform: translateY(-1px);
}

/* Markdown Separator */
hr {
    border-top: 1px solid rgba(255, 255, 255, 0.1);
    margin: 25px 0;
}

/* Chat area styling */
.gradio-chatbot {
    border: 1px solid rgba(255,255,255,0.1);
    border-radius: 12px;
    background-color: rgba(0, 0, 0, 0.15); /* Slightly transparent for depth */
    min-height: 520px; /* Ensure consistent height */
}

/* Input/Send controls */
.gradio-row {
    margin-top: 15px;
}

/* Textbox styling */
.gradio-textbox textarea {
    background-color: #2a2a40; /* Dark input field */
    color: #e0e0e0;
    border: 1px solid #4a4a6a;
    border-radius: 8px;
    padding: 10px;
    box-shadow: inset 0 1px 3px rgba(0,0,0,0.3);
}
.gradio-textbox textarea:focus {
    border-color: #a0e0ff;
    box-shadow: 0 0 0 2px rgba(160, 224, 255, 0.3);
}

/* Send button styling */
.gradio-button.primary {
    background: linear-gradient(45deg, #007bff, #00c0ff); /* Blue gradient for send */
    border: none;
    border-radius: 8px;
    color: white;
    font-weight: 600;
    transition: all 0.2s ease;
}
.gradio-button.primary:hover {
    background: linear-gradient(45deg, #0056b3, #009be6);
    transform: translateY(-2px);
    box-shadow: 0 4px 10px rgba(0,0,0,0.3);
}

/* Clear button styling */
.gradio-button {
    background-color: #dc3545; /* Red for clear */
    border: none;
    border-radius: 8px;
    color: white;
    font-weight: 600;
    transition: all 0.2s ease;
}
.gradio-button:hover {
    background-color: #c82333;
    transform: translateY(-2px);
    box-shadow: 0 4px 10px rgba(0,0,0,0.3);
}

/* Small text at bottom */
span[style*='opacity:0.7'] {
    color: #b0b0d0;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
    # Header with combined logo and text using CSS classes
    gr.HTML(
        f"""
        <div class="header-wrap">
            <img src="{LOGO_URL}" alt="Python Logo" style="height:64px; width:64px; object-fit: contain; border-radius: 8px;">
            <div>
                <div class="header-title">Python Tutor Bot</div>
                <div class="header-subtitle">
                    Ask anything about Python — concepts, debugging, best practices, and examples.
                </div>
            </div>
        </div>
        """
    )

    gr.Markdown("___")

    chatbot = gr.Chatbot(height=420)
    input_box = gr.Textbox(
        placeholder="Type your Python question and press Enter",
        show_label=False
    )
    clear_btn = gr.Button("Clear")

    messages_state = gr.State(build_initial_messages())

    input_box.submit(
        fn=chat_fn,
        inputs=[input_box, chatbot, messages_state],
        outputs=[input_box, chatbot, messages_state]
    )

    clear_btn.click(
        fn=reset_chat,
        inputs=None,
        outputs=[chatbot, messages_state]
    )

demo.launch(debug=False)

/tmp/ipython-input-3013770024.py:230: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-3013770024.py:230: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-3013770024.py:248: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)
/tmp/ipython-input-3013770024.py:248: DeprecationWarning: The defaul

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32c12121787114b455.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages


# -----------------------------
# Gradio UI (Upgraded)
# -----------------------------

FAQ_QUESTIONS = [
    "What is the difference between a list, tuple, and set in Python?",
    "How do I use dictionaries effectively in Python?",
    "What are Python functions and how do *args and **kwargs work?",
    "How does OOP work in Python (classes, objects, inheritance)?",
    "How do I handle errors using try/except?",
    "What are list comprehensions and when should I use them?",
    "How do I read and write files in Python?"
]

def set_question(q):
    return q

def clear_all():
    return [], init_messages(), ""

# Updated logo URL (RAW link)
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

css = """
/* Overall spacing + subtle polish */
#app_container {max-width: 1200px; margin: 0 auto;}

/* Header styling */
.header-wrap {
    display: flex;
    align-items: center;
    gap: 14px;
    padding: 10px 6px 2px 6px;
}
.header-title {
    font-size: 28px;
    font-weight: 700;
    line-height: 1.1;
}
.header-subtitle {
    font-size: 12.5px;
    opacity: 0.75;
    margin-top: 2px;
}

/* FAQ card-like feel */
.faq-box {
    border: 1px solid rgba(255,255,255,0.08);
    border-radius: 12px;
    padding: 14px;
}

/* Make buttons full width in FAQ column */
.faq-btn button {
    width: 100%;
    justify-content: flex-start;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
    # Header Row
    with gr.Row():
        with gr.Column(scale=1, min_width=80):
            logo = gr.Image(
                value=LOGO_URL,  # ✅ updated to raw URL
                label=None,
                show_label=False,
                show_download_button=False,
                show_share_button=False,
                height=64,
                width=64,
                container=False
            )
        with gr.Column(scale=10):
            gr.HTML(
                """
                <div class="header-wrap">
                    <div>
                        <div class="header-title">Python Tutor Bot</div>
                        <div class="header-subtitle">
                            Ask anything about Python — concepts, debugging, best practices, and examples.
                        </div>
                    </div>
                </div>
                """
            )

    gr.Markdown("---")

    # State (unchanged logic)
    state = gr.State(init_messages())

    # Two-column layout
    with gr.Row(equal_height=True):
        # LEFT: FAQ + Quick Ask
        with gr.Column(scale=4, min_width=320):
            with gr.Group(elem_classes=["faq-box"]):
                gr.Markdown("### FAQ — Most Asked Python Questions")

                gr.Markdown(
                    "Click a question to auto-fill it, then press **Enter** or click **Send**."
                )

                # Buttons for FAQ
                faq_buttons = []
                for q in FAQ_QUESTIONS:
                    b = gr.Button(q, elem_classes=["faq-btn"])
                    faq_buttons.append(b)

                gr.Markdown("### Quick prompt ideas")
                quick = gr.Radio(
                    choices=[
                        "Explain with a simple example",
                        "Give me a beginner-friendly analogy",
                        "Show common mistakes to avoid",
                        "Provide a short quiz question",
                        "Compare two approaches briefly"
                    ],
                    label="Add a style preference (optional)",
                    value=None
                )

        # RIGHT: Chat area
        with gr.Column(scale=8, min_width=520):
            chatbot = gr.Chatbot(
                height=520,
                bubble_full_width=False,
                label="Conversation"
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your Python question here…",
                    label=None,
                    scale=9
                )
                send = gr.Button("Send", variant="primary", scale=1)

            with gr.Row():
                clear = gr.Button("Clear Chat")
                gr.Markdown(
                    "<span style='opacity:0.7;font-size:12px;'>Context is preserved across turns unless you clear.</span>"
                )

    # FAQ button -> fill textbox
    for b, q in zip(faq_buttons, FAQ_QUESTIONS):
        b.click(
            fn=lambda q=q: set_question(q),
            inputs=None,
            outputs=msg
        )

    # Optional quick preference: append hint to textbox (UI-only)
    def apply_quick_pref(pref, current_text):
        if not pref:
            return current_text
        if current_text and current_text.strip():
            return f"{current_text.strip()} ({pref})"
        return pref

    quick.change(
        fn=apply_quick_pref,
        inputs=[quick, msg],
        outputs=msg
    )

    # Submit logic (unchanged)
    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    send.click(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    # Clear
    clear.click(
        fn=clear_all,
        inputs=None,
        outputs=[chatbot, state, msg]
    )

demo.launch(debug=False)


/tmp/ipython-input-1706520990.py:149: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-1706520990.py:149: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-1706520990.py:214: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1706520990.py:214: DeprecationWarning: The 'bubble_full_widt

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c8bc800b8105436c3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Steps to deploy on Hugging Face Spaces (Gradio)

Hugging Face → New Space

Choose Gradio as SDK. This sets your Space to use Gradio via the README config.
Hugging Face

Pick a name and visibility.

Add files
1. app.py (your code).

2. requirements.txt.

requirements.txt
Use something like:

gradio
openai


##Add your secret

Go to your Space Settings → Variables and secrets.

Under Secrets, add:

Name: OPENAI_API_KEY

Value: your key
Secrets are injected into the runtime as environment variables.
Hugging Face


Commit

If you’re editing in the browser, just save/commit.

The Space will rebuild and launch automatically.

In [ ]:


# ADD THIS IN APP.PY




import os
import gradio as gr
from openai import OpenAI

# -----------------------------
# Load OpenAI key from HF Secrets
# -----------------------------
# In Hugging Face Spaces, Secrets/Variables are available as environment variables.
# Add this in your Space Settings -> Variables and secrets -> Secrets
# Key name: OPENAI_API_KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



client = OpenAI(api_key=OPENAI_API_KEY)

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages


# -----------------------------
# Gradio UI (Upgraded)
# -----------------------------

FAQ_QUESTIONS = [
    "What is the difference between a list, tuple, and set in Python?",
    "How do I use dictionaries effectively in Python?",
    "What are Python functions and how do *args and **kwargs work?",
    "How does OOP work in Python (classes, objects, inheritance)?",
    "How do I handle errors using try/except?",
    "What are list comprehensions and when should I use them?",
    "How do I read and write files in Python?"
]

def set_question(q):
    return q

def clear_all():
    return [], init_messages(), ""

# Updated logo URL (RAW link)
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

css = """
/* Overall spacing + subtle polish */
#app_container {max-width: 1200px; margin: 0 auto;}

/* Header styling */
.header-wrap {
    display: flex;
    align-items: center;
    gap: 14px;
    padding: 10px 6px 2px 6px;
}
.header-title {
    font-size: 28px;
    font-weight: 700;
    line-height: 1.1;
}
.header-subtitle {
    font-size: 12.5px;
    opacity: 0.75;
    margin-top: 2px;
}

/* FAQ card-like feel */
.faq-box {
    border: 1px solid rgba(255,255,255,0.08);
    border-radius: 12px;
    padding: 14px;
}

/* Make buttons full width in FAQ column */
.faq-btn button {
    width: 100%;
    justify-content: flex-start;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
    # Header Row
    with gr.Row():
        with gr.Column(scale=1, min_width=80):
            logo = gr.Image(
                value=LOGO_URL,
                label=None,
                show_label=False,
                show_download_button=False,
                show_share_button=False,
                height=64,
                width=64,
                container=False
            )
        with gr.Column(scale=10):
            gr.HTML(
                """
                <div class="header-wrap">
                    <div>
                        <div class="header-title">Python Tutor Bot</div>
                        <div class="header-subtitle">
                            Ask anything about Python — concepts, debugging, best practices, and examples.
                        </div>
                    </div>
                </div>
                """
            )

    gr.Markdown("---")

    # State (unchanged logic)
    state = gr.State(init_messages())

    # Two-column layout
    with gr.Row(equal_height=True):
        # LEFT: FAQ + Quick Ask
        with gr.Column(scale=4, min_width=320):
            with gr.Group(elem_classes=["faq-box"]):
                gr.Markdown("### FAQ — Most Asked Python Questions")

                gr.Markdown(
                    "Click a question to auto-fill it, then press **Enter** or click **Send**."
                )

                # Buttons for FAQ
                faq_buttons = []
                for q in FAQ_QUESTIONS:
                    b = gr.Button(q, elem_classes=["faq-btn"])
                    faq_buttons.append(b)

                gr.Markdown("### Quick prompt ideas")
                quick = gr.Radio(
                    choices=[
                        "Explain with a simple example",
                        "Give me a beginner-friendly analogy",
                        "Show common mistakes to avoid",
                        "Provide a short quiz question",
                        "Compare two approaches briefly"
                    ],
                    label="Add a style preference (optional)",
                    value=None
                )

        # RIGHT: Chat area
        with gr.Column(scale=8, min_width=520):
            chatbot = gr.Chatbot(
                height=520,
                bubble_full_width=False,
                label="Conversation"
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your Python question here…",
                    label=None,
                    scale=9
                )
                send = gr.Button("Send", variant="primary", scale=1)

            with gr.Row():
                clear = gr.Button("Clear Chat")
                gr.Markdown(
                    "<span style='opacity:0.7;font-size:12px;'>Context is preserved across turns unless you clear.</span>"
                )

    # FAQ button -> fill textbox
    for b, q in zip(faq_buttons, FAQ_QUESTIONS):
        b.click(
            fn=lambda q=q: set_question(q),
            inputs=None,
            outputs=msg
        )

    # Optional quick preference: append hint to textbox (UI-only)
    def apply_quick_pref(pref, current_text):
        if not pref:
            return current_text
        if current_text and current_text.strip():
            return f"{current_text.strip()} ({pref})"
        return pref

    quick.change(
        fn=apply_quick_pref,
        inputs=[quick, msg],
        outputs=msg
    )

    # Submit logic (unchanged)
    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    send.click(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    # Clear
    clear.click(
        fn=clear_all,
        inputs=None,
        outputs=[chatbot, state, msg]
    )

demo.launch(debug=False)
